This script is used to pull NWM data of different formats and reorganize it into a single CSV.


In [1]:
import pandas as pd
import numpy as np
from prep_input_data import read_modeled_estimates
from pywr_drb_node_data import nwm_site_matches

start_date = '1983/10/01'
end_date = '2016/12/31'

# Directories
fig_dir = 'figs/'
input_dir = 'input_data/'
weap_dir = input_dir + 'WEAP_23Aug2022_gridmet/'

# Constants
cms_to_mgd = 22.82
cm_to_mg = 264.17/1e6
cfs_to_mgd = 0.0283 * 22824465.32 / 1e6


In [2]:
# Read NWM data from different sources
nwm_gauge_flow = read_modeled_estimates(f'{input_dir}modeled_gages/streamflow_daily_nwmv21.txt',
                                    '\t', 'UTC_date', 'site_no', 'q_cms_mod', start_date, end_date)
nwm_lake_flow = read_modeled_estimates(f'{input_dir}modeled_gages/lakes_daily_1979_2020_nwmv21.csv',
                                        ',', 'UTC_date', 'feature_id', 'inflow', start_date, end_date)
nwm_segment_flow = read_modeled_estimates(f'{input_dir}modeled_gages/streamflow_daily_1979_2020_nwmv21_reaches.csv',
                                        ',', 'UTC_date', 'link', 'q_cms', start_date, end_date)
nwm_lake_flow.columns = [str(i) for i in nwm_lake_flow.columns]
nwm_segment_flow.columns = [str(i) for i in nwm_segment_flow.columns]

In [3]:
all_nwm_data = pd.concat([nwm_gauge_flow, nwm_lake_flow, nwm_segment_flow], axis=1)
all_nwm_data.shape

(12146, 99)

In [4]:
drb_nwm_flows = pd.DataFrame(index = pd.date_range(start_date, end_date), columns = np.array([*nwm_site_matches.values()]).flatten())
for node, site_id in nwm_site_matches.items():
    drb_nwm_flows.loc[:, site_id] = all_nwm_data.loc[:, site_id]
drb_nwm_flows.mean()

2613174       451.423320
1748473       357.737843
4146742       122.742336
2741600       209.484510
2739068        59.378525
120052035      60.046894
4148582       188.054188
4186689        90.556483
4185065       376.364616
2588031         0.720830
2585287        10.477591
2591099         9.859893
2589015         2.234628
4779981       173.490286
4778721         3.850229
4782813       163.013342
4780087        64.118252
4648728        18.061543
01425000      454.044604
01417000      360.721285
2617364      1698.217456
01436000      129.858166
01433500      198.124534
4151628      3627.969296
01449800       91.556837
01447800      378.343854
2590277      7502.417178
01463620       21.887486
2590137        98.149466
01470960      163.697153
4784841      2066.623573
4652144       268.574146
dtype: float64

In [6]:
drb_nwm_flows.to_csv('./input_data/modeled_gages/streamflow_daily_nwmv21_mgd.csv', sep=',')